In [ ]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../scripts")
import file_management as fm
import lib_masks as masks

### NNFit precut from [wiki](http://antares-wiki.in2p3.fr/doku.php?id=nnfit)

In [ ]:
def nnfit_precut(df):    
    cut_r_closest = df['NNFitShower_RVertex'] < 50
    cut_z_closest = np.abs(df['NNFitShower_ZVertex']) < 150
    cut_sigma_dir = df['NNFitShower_SigmaTheta'] < 14.9
    
    return cut_r_closest & cut_z_closest & cut_sigma_dir

In [ ]:
def is_bbfit_reconstructed(df):
    return df['bbfit_flag'] == True

In [ ]:
def is_bbfit_single_line_event(df):
    return (df['bbfit_zenith_deg'].notna()) & (df['bbfit_azimuth_deg'].isna())

In [ ]:
def not_reconstructed_by_nnfit(df):
    return df['NNFitShower_RVertex'].isna()

## Check which columns you want to add

In [ ]:
COLUMNS = [
        "RunID",
        "EventID",
        "TriggCounter",
        "Frame",
        "interaction_type",
        "is_cc",
        "Type",
        "bbfit_flag",
        "bbfit_zenith_deg",
        "bbfit_azimuth_deg"
    ]

## Add the extracted AntDST files 

In [ ]:
cluster = "woody"

if cluster == "woody":
    path = "/home/wecapstor3/capn/mppi133h/ANTARES/mc/cut_selection/low_energy"
elif cluster == "cca":
    path = "/home/sps/km3net/users/mchadoli/ANTARES/low_energy"

In [ ]:
df = fm.rootfile_to_df(os.path.join(path,"full_nutau_sample.root"), columns = COLUMNS)

## Load the NNFit files

In [ ]:
nnfit_path = "/home/wecapstor3/capn/mppi133h/ANTARES/mc/nnfit_reco/"

# Load nnfit reco files
nnfit_files = fm.list_files_with_pattern(nnfit_path, f"*taus*")

df_nnfit = fm.load_dataframes(nnfit_files, folder_path= nnfit_path)

In [ ]:
def rename_h5_df_cols(
    df,
    mapper={
        "TrigCount": "TriggCounter",
        "EventID": "Frame",
    },
):
    return df.rename(columns=mapper)

## Merge the two dataframes

In [ ]:
print("Renaming the columns...")
df_nnfit = rename_h5_df_cols(df_nnfit)

df_merged = df.merge(df_nnfit, on=["RunID","Frame","TriggCounter"], how="left")

print("Number of events in the merged dataframe: ", df_merged.shape[0])
print("Number of NNfit events:", df_nnfit.shape[0])
print("Number of nutau events:", df.shape[0])

In [ ]:
total_events = df_merged.shape[0]
total_events

### Clean events not reconstructed by BBFit

In [ ]:
df_merged = df_merged[is_bbfit_reconstructed(df_merged)]

In [ ]:
print("Number of events after bbfit reconstruction: ", df_merged.shape[0])
print("Percentage from total generated events: ", df_merged.shape[0]/total_events * 100)

In [ ]:
print("Number of single line events: ", df_merged[is_bbfit_single_line_event(df_merged)].shape[0])
print("Percentage from total generated events: ", df_merged[is_bbfit_single_line_event(df_merged)].shape[0]/total_events * 100)

In [ ]:
single_line_events = df_merged[is_bbfit_single_line_event(df_merged)]

In [ ]:
single_line_events.head(15)

In [ ]:
missing_events = single_line_events[not_reconstructed_by_nnfit(single_line_events)]

In [ ]:
missing_events.head(15)